# Analyze Our Default Simulations

In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

import fates_calibration_library.analysis_functions as analysis
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.plotting_functions as plotting
import fates_calibration_library.utils as utils

In [2]:
# Setup PBSCluster
cluster = PBSCluster(
    cores=1,                                                   # The number of cores you want
    memory='25GB',                                             # Amount of memory
    processes=1,                                               # How many processes
    queue='casper',                                            # The type of queue to utilize
    local_directory='/glade/work/afoster',                     # Use your local directory
    resource_spec='select=1:ncpus=1:mem=25GB',                 # Specify resources
    log_directory='/glade/derecho/scratch/afoster/dask_logs',  # log directory
    account='P08010000',                                       # Input your project ID here
    walltime='04:00:00',                                       # Amount of wall time
    interface='ext')

In [3]:
cluster.scale(30)
dask.config.set({
    'distributed.dashboard.link': 'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
})
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.175:42223,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/8787/status,Total threads: 0
Started: 1 minute ago,Total memory: 0 B


## Set Up

First specify history variables and variables to investigate

In [4]:
fates_vars = ['FATES_FRACTION', 'FATES_GPP', 'FATES_LAI', 'QVEGE',
              'QVEGT', 'EFLX_LH_TOT', 'FSH', 'FSR', 'FSDS', 'FSA',
              'FIRE', 'FLDS', 'FCTR', 'FCEV', 'FGEV', 'BTRANMN',
              'FGR', 'SOILWATER_10CM', 'TWS', 'QRUNOFF', 'SNOWDP',
              'TV', 'TG', 'TSA', 'TBOT']

clm_vars = ['FPSN', 'TLAI', 'QVEGE', 'QVEGT', 'EFLX_LH_TOT', 'FSH',
            'FSR', 'FSDS', 'FSA', 'FIRE', 'FLDS', 'FCTR', 'FCEV',
            'FGEV', 'BTRANMN', 'FGR', 'SOILWATER_10CM', 'TWS',
            'QRUNOFF', 'SNOWDP', 'TV', 'TG', 'TSA', 'TBOT']

out_vars = ['GPP', 'LAI', 'EFLX_LH_TOT', 'FSH', 'EF', 'SOILWATER_10CM', 'ASA',
            'FSR', 'FSA', 'FIRE', 'RLNS', 'RN', 'BTRANMN', 'TV']

In [7]:
client.shutdown()

### Files Needed

In [5]:
# sparse grid clustering file
grid_dir = '/glade/work/afoster/FATES_calibration/surfdata'
grid_file = 'clusters.clm51_PPEn02ctsm51d021_2deg_GSWP3V1_leafbiomassesai_PPE3_hist.annual+sd.400.nc'
sparse_grid = xr.open_dataset(os.path.join(grid_dir, grid_file))

# sparse grid whittaker biomes
whittaker_ds = xr.open_dataset('/glade/work/afoster/FATES_calibration/observations/whittaker/whitkey.nc')

# fetch the sparsegrid landarea - needed for unit conversion
land_area_file = '/glade/work/afoster/FATES_calibration/CLM5PPE/postp/sparsegrid_landarea.nc'
land_area = xr.open_dataset(land_area_file).landarea

# create a global land frac and area grid
land_frac_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")
target_grid = analysis.create_target_grid(land_frac_ds, 'FSR')


# config file with conversion information in it
var_config = '/glade/work/afoster/FATES_calibration/fates_calibration_library/configs/model_conversion.yaml'
var_dict = utils.get_config_file(var_config)

# ilamb observations
ilamb_config_file = '/glade/work/afoster/FATES_calibration/fates_calibration_library/configs/ilamb_conversion.yaml'
ilamb_config = utils.get_config_file(ilamb_config_file)
ilamb_obs = xr.open_dataset('/glade/work/afoster/FATES_calibration/observations/all_ILAMB_obs.nc')

## Read in Datasets

In [6]:
# FATES sparse grid run
sparse_fates_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_fates_sparse_vai_default_latest/lnd/hist'
sparse_fates_ds = analysis.post_process_ds(sparse_fates_dir, fates_vars, whittaker_ds,
                                     years=[2000, 2015], run_dict={'filter_nyears': 20})

AttributeError: 'Dataset' object has no attribute 'SNOW'

In [ ]:
# FATES sparse grid run - latest tag
sparse_fates_dir_latest = '/glade/derecho/scratch/afoster/archive/ctsm60SP_fates_sparse_crujra/lnd/hist'
sparse_fates_ds_latest = analysis.post_process_ds(sparse_fates_dir_latest, fates_vars, whittaker_ds,
                                     years=[2000, 2015], run_dict={'filter_nyears': 20})

In [ ]:
# FATES global run
global_fates_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_fates_full/lnd/hist'
global_fates_ds = analysis.post_process_ds(global_fates_dir, fates_vars, whittaker_ds,
                                     [2000, 2015],
                                     {'sparse': False, 'filter_nyears': 20})

In [ ]:
# CLM sparse grid run
sparse_clm_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_bigleaf_sparse_hydrstress/lnd/hist'
sparse_clm_ds = analysis.post_process_ds(sparse_clm_dir, clm_vars, whittaker_ds,
                                     [2000, 2015],
                                     {'sparse': True, 'fates': False, 'filter_nyears': 20})

## Post-Process Datasets


### Temporal Averaging
First calculate annual and monthly means

In [ ]:
# annual means
fates_ann_means_sparse = analysis.apply_to_vars(sparse_fates_ds, out_vars, func=analysis.calculate_annual_mean, add_sparse=True,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars},
                                         new_units={var: var_dict[var]["annual_units"] for var in out_vars}).mean(dim='year')

fates_ann_means_sparse_latest = analysis.apply_to_vars(sparse_fates_ds_latest, out_vars, func=analysis.calculate_annual_mean, add_sparse=True,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars},
                                         new_units={var: var_dict[var]["annual_units"] for var in out_vars}).mean(dim='year')

fates_ann_means_glob = analysis.apply_to_vars(global_fates_ds, out_vars,
                                         func=analysis.calculate_annual_mean,
                                         add_sparse=False,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars},
                                         new_units={var: var_dict[var]["annual_units"] for var in out_vars}).mean(dim='year')

clm_ann_means_sparse = analysis.apply_to_vars(sparse_clm_ds, out_vars, func=analysis.calculate_annual_mean, add_sparse=True,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars},
                                         new_units={var: var_dict[var]["annual_units"] for var in out_vars}).mean(dim='year')

In [ ]:
# monthly means
fates_month_means_sparse = analysis.apply_to_vars(sparse_fates_ds, out_vars,
                                         func=analysis.calculate_monthly_mean,
                                         add_sparse=True,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars})

fates_month_means_sparse_latest = analysis.apply_to_vars(sparse_fates_ds_latest, out_vars,
                                         func=analysis.calculate_monthly_mean,
                                         add_sparse=True,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars})

fates_month_means_glob = analysis.apply_to_vars(global_fates_ds, out_vars,
                                         func=analysis.calculate_monthly_mean,
                                         add_sparse=False,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars})

clm_month_means_sparse = analysis.apply_to_vars(sparse_clm_ds, out_vars,
                                         func=analysis.calculate_monthly_mean,
                                         add_sparse=True,
                                         conversion_factor={var: var_dict[var]["time_conversion_factor"] for var in out_vars})

### Calculate Month of Max
Calculate the month of the maximum variable

In [ ]:
# month of max
fates_max_month_sparse = analysis.apply_to_vars(fates_month_means_sparse, out_vars,
                                         func=lambda x: x.idxmax(dim="month"),
                                         add_sparse=True)

fates_max_month_sparse_latest = analysis.apply_to_vars(fates_month_means_sparse_latest, out_vars,
                                         func=lambda x: x.idxmax(dim="month"),
                                         add_sparse=True)

fates_max_month_glob = analysis.apply_to_vars(fates_month_means_glob, out_vars,
                                         func=lambda x: x.idxmax(dim="month"),
                                         add_sparse=False)

clm_max_month_sparse = analysis.apply_to_vars(clm_month_means_sparse, out_vars,
                                         func=lambda x: x.idxmax(dim="month"),
                                         add_sparse=True)

### Remap Sparse Grid

Remap the annual and monthly sparse means to a global grid

In [ ]:
# maps from the annual means
fates_sparse_glob = analysis.get_sparse_maps(fates_ann_means_sparse, sparse_grid,
                                     out_vars)

fates_sparse_glob_latest = analysis.get_sparse_maps(fates_ann_means_sparse_latest, sparse_grid,
                                     out_vars)

clm_sparse_glob = analysis.get_sparse_maps(clm_ann_means_sparse, sparse_grid,
                                     out_vars)


# maps from the monthly maxes
fates_sparse_glob_month_max = analysis.get_sparse_maps(fates_max_month_sparse, sparse_grid,
                                     out_vars)

fates_sparse_glob_month_max_latest = analysis.get_sparse_maps(fates_max_month_sparse_latest, sparse_grid,
                                     out_vars)

clm_sparse_glob_month_max = analysis.get_sparse_maps(clm_max_month_sparse, sparse_grid,
                                     out_vars)

### Zonal Means
Calculate zonal means (by latitude)

In [ ]:
# zonal means
fates_zonal_sparse = analysis.apply_to_vars(fates_sparse_glob, out_vars,
                                         func=analysis.calculate_zonal_mean,
                                         add_sparse=False,
                                         land_area=target_grid.land_area,
                                         conversion_factor={var: var_dict[var]["area_conversion_factor"] for var in out_vars})

fates_zonal_sparse_latest = analysis.apply_to_vars(fates_sparse_glob_latest, out_vars,
                                         func=analysis.calculate_zonal_mean,
                                         add_sparse=False,
                                         land_area=target_grid.land_area,
                                         conversion_factor={var: var_dict[var]["area_conversion_factor"] for var in out_vars})

fates_zonal_global = analysis.apply_to_vars(fates_ann_means_glob, out_vars,
                                         func=analysis.calculate_zonal_mean,
                                         add_sparse=False,
                                         land_area=target_grid.land_area,
                                         conversion_factor={var: var_dict[var]["area_conversion_factor"] for var in out_vars})

clm_zonal_sparse = analysis.apply_to_vars(clm_sparse_glob, out_vars,
                                         func=analysis.calculate_zonal_mean,
                                         add_sparse=False,
                                         land_area=target_grid.land_area,
                                         conversion_factor={var: var_dict[var]["area_conversion_factor"] for var in out_vars})

### Climatology
Calculate climatology

In [ ]:
# area means from monthly
fates_sparse_glob_monthly = analysis.get_sparse_area_means(fates_month_means_sparse, 'global',
                                                out_vars, var_dict, land_area,
                                                sparse_fates_ds.biome)

fates_sparse_glob_monthly_latest = analysis.get_sparse_area_means(fates_month_means_sparse_latest, 'global',
                                                out_vars, var_dict, land_area,
                                                sparse_fates_ds.biome)

fates_glob_monthly = analysis.apply_to_vars(fates_month_means_glob, out_vars,
                                         func=analysis.area_mean,
                                         add_sparse=False,
                                         cf={var: var_dict[var]["area_conversion_factor"] for var in out_vars},
                                         land_area=target_grid.land_area)

clm_sparse_glob_monthly = analysis.get_sparse_area_means(clm_month_means_sparse, 'global',
                                                out_vars, var_dict, land_area,
                                                sparse_clm_ds.biome)

### Calculate Global Annual Values

Area-weighted averages

In [ ]:
# sparse area means
fates_area_means_sparse = analysis.get_sparse_area_means(fates_ann_means_sparse,
                                            'global', out_vars,
                                            var_dict, land_area, sparse_fates_ds.biome)

fates_area_means_sparse_latest = analysis.get_sparse_area_means(fates_ann_means_sparse_latest,
                                            'global', out_vars,
                                            var_dict, land_area, sparse_fates_ds.biome)

clm_area_means_sparse = analysis.get_sparse_area_means(clm_ann_means_sparse,
                                            'global', out_vars,
                                            var_dict, land_area, sparse_clm_ds.biome)

## How Well the Does the Sparse Grid Match Global Simulations
### Annual Maps

In [ ]:
var = 'GPP'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'old',
                             'new', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'LAI'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'EFLX_LH_TOT'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'FSH'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'EF'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'ASA'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'SOILWATER_10CM'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

In [ ]:
var = 'BTRANMN'
plotting.plot_two_model_diff(fates_sparse_glob[var], fates_sparse_glob_latest[var], 'Old',
                             'New', var, var_dict[var]['annual_units'],
                             var_dict[var]['cmap'])

### Zonal Means

In [ ]:
var_name = 'GPP'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'LAI'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'ASA'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'EFLX_LH_TOT'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'FSH'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'EF'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'SOILWATER_10CM'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

In [ ]:
var_name = 'BTRANMN'
plotting.plot_zonal_mean_diff([fates_zonal_sparse[var_name], fates_zonal_sparse_latest[var_name]],
                              ['Old', 'New'], var_name, var_dict[var_name]['long_name'],
                              var_dict[var_name]['global_units'])

### Month of Max

In [ ]:
var = 'GPP'
plotting.plot_month_of_max_diff(fates_sparse_glob_month_max[var],
                                fates_sparse_glob_month_max_latest[var], 'Old', 'New', var)

In [ ]:
var = 'EFLX_LH_TOT'
plotting.plot_month_of_max_diff(fates_sparse_glob_month_max[var],
                                fates_sparse_glob_month_max_latest[var], 'Sparse', 'Global', var)

In [ ]:
var = 'FSH'
plotting.plot_month_of_max_diff(fates_sparse_glob_month_max[var],
                                fates_sparse_glob_month_max_latest[var], 'Sparse', 'Global', var)

In [ ]:
var = 'EF'
plotting.plot_month_of_max_diff(fates_sparse_glob_month_max[var],
                                fates_sparse_glob_month_max_latest[var], 'Sparse', 'Global', var)

In [ ]:
var = 'ASA'
plotting.plot_month_of_max_diff(fates_sparse_glob_month_max[var],
                                fates_sparse_glob_month_max_latest[var], 'Sparse', 'Global', var)

In [ ]:
var = 'SOILWATER_10CM'
plotting.plot_month_of_max_diff(fates_sparse_glob_month_max[var],
                                fates_sparse_glob_month_max_latest[var], 'Sparse', 'Global', var)

### Climatology

In [ ]:
var = 'GPP'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "CRU-JRA", "GSWP3", var, var, var_dict[var]['global_units'])

In [ ]:
var = 'EFLX_LH_TOT'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "CRU-JRA", "GSWP3", var, var, var_dict[var]['global_units'])

In [ ]:
var = 'FSH'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "CRU-JRA", "GSWP3", var, var, var_dict[var]['global_units'])
plt.savefig('cru_vs_gswp3_fsh.png')

In [ ]:
var = 'EF'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "New", "Old", var, var, var_dict[var]['global_units'])

In [ ]:
var = 'ASA'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "New", "Old", var, var, var_dict[var]['global_units'])

In [ ]:
var = 'SOILWATER_10CM'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "New", "Old", var, var, var_dict[var]['global_units'])

In [ ]:
var = 'BTRANMN'
plotting.plot_annual_cycle_diff(fates_sparse_glob_monthly_latest[var], fates_sparse_glob_monthly[var],
                                "New", "Old", var, var, var_dict[var]['global_units'])

### Global Annual Values

In [ ]:
summary_df = plotting.summarize_differences(fates_area_means_sparse, fates_area_means_sparse_latest,
                                            'Old', 'New', var_dict)

In [ ]:
plotting.plot_heatmap(summary_df)

## Observational Comparisons
### Annual Maps

In [ ]:
var = 'GPP'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var]['var'], ilamb_config[var]['models'])
plotting.plot_model_obs_diff(fates_sparse_glob_latest, ilamb_var, target_grid.landfrac, var,
                             var_dict[var]['annual_units'], 'FATES')

In [ ]:
var = 'LAI'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var]['var'], ilamb_config[var]['models'])
plotting.plot_model_obs_diff(fates_sparse_glob_latest, ilamb_var, target_grid.landfrac, var,
                             var_dict[var]['annual_units'], 'FATES')

In [ ]:
var = 'ASA'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var]['var'], ilamb_config[var]['models'])
plotting.plot_model_obs_diff(fates_sparse_glob_latest, ilamb_var, target_grid.landfrac, var, var_dict[var]['annual_units'], 'FATES')

In [ ]:
var = 'EFLX_LH_TOT'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var]['var'], ilamb_config[var]['models'])
plotting.plot_model_obs_diff(fates_sparse_glob_latest, ilamb_var, target_grid.landfrac, var, var_dict[var]['annual_units'], 'FATES')

In [ ]:
var = 'FSH'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var]['var'], ilamb_config[var]['models'])
plotting.plot_model_obs_diff(fates_sparse_glob_latest, ilamb_var, target_grid.landfrac, var, var_dict[var]['annual_units'], 'FATES')

In [ ]:
var = 'EF'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var]['var'], ilamb_config[var]['models'])
plotting.plot_model_obs_diff(fates_sparse_glob_latest, ilamb_var, target_grid.landfrac, var, var_dict[var]['annual_units'], 'FATES')

### Zonal Means

In [ ]:
var_name = 'GPP'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var_name]['var'], ilamb_config[var_name]['models'])
plotting.plot_model_obs_zonal_diff(ilamb_var, fates_zonal_sparse_latest[var_name], ilamb_obs.land_area,
                          ilamb_config[var_name]['conversion_factor'],
                          var_name, var_name, var_dict[var]['global_units'])

In [ ]:
var_name = 'LAI'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var_name]['var'], ilamb_config[var_name]['models'])
plotting.plot_model_obs_zonal_diff(ilamb_var, fates_zonal_sparse_latest[var_name], ilamb_obs.land_area,
                          ilamb_config[var_name]['conversion_factor'],
                          var_name, var_name, var_dict[var]['global_units'])

In [ ]:
var_name = 'ASA'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var_name]['var'], ilamb_config[var_name]['models'])
plotting.plot_model_obs_zonal_diff(ilamb_var, fates_zonal_sparse_latest[var_name], ilamb_obs.land_area,
                          ilamb_config[var_name]['conversion_factor'],
                          var_name, var_name, var_dict[var]['global_units'])

In [ ]:
var_name = 'EFLX_LH_TOT'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var_name]['var'], ilamb_config[var_name]['models'])
plotting.plot_model_obs_zonal_diff(ilamb_var, fates_zonal_sparse_latest[var_name], ilamb_obs.land_area,
                          ilamb_config[var_name]['conversion_factor'],
                          var_name, var_name, var_dict[var]['global_units'])

In [ ]:
var_name = 'FSH'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var_name]['var'], ilamb_config[var_name]['models'])
plotting.plot_model_obs_zonal_diff(ilamb_var, fates_zonal_sparse_latest[var_name], ilamb_obs.land_area,
                          ilamb_config[var_name]['conversion_factor'],
                          var_name, var_name, var_dict[var]['global_units'])

In [ ]:
var_name = 'EF'
ilamb_var = ilamb.get_model_da(ilamb_obs, ilamb_config[var_name]['var'], ilamb_config[var_name]['models'])
plotting.plot_model_obs_zonal_diff(ilamb_var, fates_zonal_sparse_latest[var_name], ilamb_obs.land_area,
                          ilamb_config[var_name]['conversion_factor'],
                          var_name, var_name, var_dict[var]['global_units'])

### Climatology

In [ ]:
var_name = 'GPP'
ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}_cycle", ilamb_config[var_name]['models'])
plotting.plot_model_obs_climatology_diff(ilamb_var, fates_sparse_glob_monthly_latest[var_name], var_name, var_name, var_dict[var_name]['global_units'])

In [ ]:
var_name = 'LAI'
ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}_cycle", ilamb_config[var_name]['models'])
plotting.plot_model_obs_climatology_diff(ilamb_var, fates_sparse_glob_monthly_latest[var_name], var_name, var_name, var_dict[var_name]['global_units'])

In [ ]:
var_name = 'ASA'
ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}_cycle", ilamb_config[var_name]['models'])
plotting.plot_model_obs_climatology_diff(ilamb_var, fates_sparse_glob_monthly_latest[var_name], var_name, var_name, var_dict[var_name]['global_units'])

In [ ]:
var_name = 'EFLX_LH_TOT'
ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}_cycle", ilamb_config[var_name]['models'])
plotting.plot_model_obs_climatology_diff(ilamb_var, fates_sparse_glob_monthly_latest[var_name], var_name, var_name, var_dict[var_name]['global_units'])

In [ ]:
var_name = 'FSH'
ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}_cycle", ilamb_config[var_name]['models'])
plotting.plot_model_obs_climatology_diff(ilamb_var, fates_sparse_glob_monthly_latest[var_name], var_name, var_name, var_dict[var_name]['global_units'])

In [ ]:
var_name = 'EF'
ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}_cycle", ilamb_config[var_name]['models'])
plotting.plot_model_obs_climatology_diff(ilamb_var, fates_sparse_glob_monthly_latest[var_name], var_name, var_name, var_dict[var_name]['global_units'])

### Global Annual Values

In [ ]:
ds_out = xr.Dataset()
for var_name in [var for var in out_vars if var not in ['SOILWATER_10CM', 'BTRANMN', 'TV']]:
    ilamb_var = ilamb.get_model_da(ilamb_obs, f"{ilamb_config[var_name]['var']}", ilamb_config[var_name]['models'])
    ds_out[var_name] = analysis.area_mean(ilamb_var, ilamb_config[var_name]['conversion_factor'], ilamb_obs.land_area).mean(dim='model')

In [ ]:
summary_df = plotting.summarize_differences(ds_out, fates_area_means_sparse_latest, 'ILAMB', 'FATES', var_dict)
plotting.plot_heatmap(summary_df)

In [ ]:
summary_df = plotting.summarize_differences(ds_out, fates_area_means_sparse, 'ILAMB', 'GSWP3', var_dict)
plotting.plot_heatmap(summary_df)

In [ ]:
summary_df = plotting.summarize_differences(fates_area_means_sparse, fates_area_means_sparse_latest, 'GSWP3', 'CRU-JRA', var_dict)
plotting.plot_heatmap(summary_df)
plt.savefig('cru_vs_gswp3_all.png')